## COMPSCI 361 Assignment 2 by Kejun Dai

## **Part 1** : Report

### **Part 1.1**: Problem motivation and Evaluation Method

The machine learning problem for this challenge is given a paragraph of an abstract of a protein, the model needs to be able to predict the whether the protein fall into category of Archaea, Bacteria, Eukaryota, or Virus. To solve the problem. I plan to use Naive Bayes algorithm to train a classifier. 

In this experiment, I will randomly split the entries in the data set into training sets and testing set beforehand. Traning set is roughly 70% of original data sets and testing set is roughly 30% of original data sets. Entry in training sets will only be used to train models and Entry in testing sets will only be used to test models. I will use Bootstrap to sample the traing data and testing data from traing set and testing set respectively

In order to test and evaluate my Naive Bayes algorithm, I plan to first run a ten-fold cross validation on the algorithm, calculating its average training accuracy. After that I put the model against testing with 20 randomly bootstrapped data sets to get its average testing accuracy. To compare two iterations of my Naive Bayes algorithm I will use t-test with `ttest_ind` from `scipy.stats` library to evaluate the statistical difference. I will accept an iteration of Naive Bayes algorithm performs better than its predecessor only if its average testing accuracy is greater and it's statistical different at 95% confident level.

### **Part 1.2.1**: Standard Naive Bayes Algorithm

My first Naive Bayes algorithm is to implement the standard Naive Bayes outlined in the Mitchell's "Machine Learning". 

In this algorithm, each abstract is regarded as a combination of occurances of different identically distributied words. These words are assumed to have the equal likelihood to appear in an abstract. The classes are represented by "A", "B","E" ,"V" repectively, and I use `[{class of A},{class of B},{class of E},{class of V}]` through out my algorithm to represent the same values that is related with each classes respectively.

The implementation will consist of three parts,`word_weight`,`label_prob` and `total_word_count`. `word_weight` is a dictionary that record the numbers of occurence of each word in abstract with different classes. `label_prob` is a array that records the prior likelihood of the abstract being certain class. `total_word_count` is a array that record the total number of occurances of words in abstracts with different classes

To train such model, the algorithm will for each entry identify the class of the abstract and increment the number of occurences of said class. It then reads through the abstract. For each occurence of word it increments the total number of occurances of words in entry's class and the number of occucurance of said word in entry class. After reading through all entries, the algorithm will calculate the prior likelihood of the abstract being certain class in log space according number of occurences of classes. It produces the model with all these informations.

When predicting the class of a given abstract, the model will first set their current predicted likelihood of each class to be the values in `label_prob` before reading through the abstract. For each word in the abstract, the model will calculatethe likelihood of occurence of these words given certain class with laplace smoothing in log space and increment them to the current predicted likelihood of said class. After reading the abstract, the model will choose one of classes with the greatest likelihood to be its prediction

### **Part 1.2.2**: Standard Naive Bayes Algorithm with oversampling minority classes

After running some test on my first Naive Bayes algorithm, I have noticed that its accuracy to predict abstracts that are Archaea or Virus abyssmal compared to predict abstracts that are Bacteria and Eukaryota (38% and 0% compared to 93% and 99%). And after analysing the training datas, I found that the data set is heavily unbalanced with majority classes Bacteria and Eukaryota take 94% of the data, while the minority classes Archaea or Virus take 6% of the data.

So I use `RandomOverSampler` from the library `imblearn.over_sampling`to oversample the minority classes so that both majority classes and minority classes take 50% of the training data before I use them to train my standard Naive Bayes Algorithm.

### **Part 1.2.3**: Complement Naive Bayes Algorithm with oversampling minority classes

In Rennie at al.(2003) "Tackling the Poor Assumptions of Naive Bayes Text Classifiers", the article idetifies the problems of the Naive Bayes Classifer on documentation problem. One of them is that the classifer will make tilted prediction even when the classes in the training data is a little bit imbalanced. The article propose Complement Naive Bayes Algorithm where it use likelihood of occurance of word in abstracts that is not the class to calculate its prediction.

Since my model's accuracy to predict abstracts that are Archaea or Virus is still much poorer than the accuracy to predict abstracts that are Bacteria and Eukaryota even after employing oversampling minority classes (48% and 76% compared to 92% and 99%). I need a way to reduce the Variance of my model in predicting abstracts that are Archaea or Virus due to oversampling almost 20 times size of the orignial data. So on atop of that I modify my standard Naive Bayes algorithm to be an implementation of Complement Naive Bayes algorithm in Rennie at al.(2003). For each word the model read in the abstract, the model will deduct the likelihood of the occurance of the word in abstracts of other classes from the predicited likelihood of classes now

### **Part 1.3**: The result and the final chosen model

In this section, I will use acronym *SNB* to describe Standard Naive Bayes (Part 1.2.1), *SNBwOM* to describe Standard Naive Bayes with Oversampling Minority classes (Part 1.2.2) and *CNBwOM* to describe Complemtary Naive Bayes with Oversampling Minority Classes (Part 1.2.3).

The reuslt of the models is in the following:
| Models | Average Training Accuracy | Average Testing Accuracy |
| --- | --- | --- |
| *SNB* | 95.56% | 91.88% |
| *SNBwOM* | 98.81% | 93.53% |
| *CNBwOM* | 98.45% | 95.31% |

It can be seen that *SNBwOM* brings great improvements on the *SNB* models where both average training accuracy and average testing accuracy has increased compared to SNB. *CNBwOM* offers an improvement of Variance on *SNBwOM* where it increase the average Testing accuracy with a tiny bit drop off from the average training accuracy. And at 95% confident level, testing accuracy of *SNBwOM* is statistically different with testing accuracy of *SNB* and testing accuracy of *CNBwOM* is statisitcally different with testing accuracy of *SNBwOM* and testing accuracy of *SNB*. We can say *CNBwOM* performs the best among three models while *SMBwOM* also performs better than *SNB*

In conclusion, *CNBwOM* will be the model I use to tackle the challenge.

---

## **Part 2** : Code Implementation

### **Part 2.1**: Implement a standard version of Naive Bayes algorithm on the dataset


In [1]:
import csv
from random import *
import pandas as pd
import math
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import ttest_ind

In [2]:
#Classes for the  abstract
possible_label=["A","B","E","V"]

#Reading training examples from the file
dataset=[]
file=open("trg.csv")
data_csv=csv.DictReader(file)
for row in data_csv:
   dataset.append([row["abstract"],row["class"]])
file.close()
print(f"There are {len(dataset)} examples in the original dataset")

#Spliting examples into training set and testing set randomly
testset=[]
trainset=[]
seed(100)
for i in dataset:
    if random()<=0.3:
        testset.append(i)
    else:
        trainset.append(i)
print(f"There are {len(trainset)} examples in the training set")
print(f"There are {len(testset)} examples in the testing set")

There are 4000 examples in the original dataset
There are 2808 examples in the training set
There are 1192 examples in the testing set


In [3]:
#Helper method to extract class distribution of a dataset
def analyze_label_distribution(dataset):
    label_count=[0,0,0,0]
    for d in dataset:
        label_count[possible_label.index(d[1])]+=1
    print(pd.DataFrame([label_count,[round(x/sum(label_count),2) for x in label_count]],columns=possible_label,index=["counts","distributions"]))

print("The training set has the following classes distribution:")
analyze_label_distribution(trainset)
print("\nThe training set has the following classes distribution:")
analyze_label_distribution(testset)

The training set has the following classes distribution:
                   A       B        E      V
counts         85.00  1122.0  1508.00  93.00
distributions   0.03     0.4     0.54   0.03

The training set has the following classes distribution:
                   A      B       E      V
counts         43.00  480.0  636.00  33.00
distributions   0.04    0.4    0.53   0.03


In [4]:
#The Standard Naive Bayesian model
class BayseianModel:
    def __init__(self,word_weight,label_prob,total_word_count) -> None:
        self.__word_weight=word_weight
        self.__label_prob=label_prob
        self.__total_word_count=total_word_count
    
    @property
    def word_weight(self):
        return self.__word_weight
    
    @property
    def label_prob(self):
        return self.__label_prob
    
    @property
    def total_word_count(self):
        return self.__total_word_count

    def predict(self,abstract:str):
        word_amount=len(self.word_weight)
        word_list=abstract.split()
        propbability=list(self.label_prob)
        for w in word_list:
            if w in self.word_weight:
                index=0
                while index<4:
                    smooth_prob=(self.word_weight[w][index]+1)/(self.total_word_count[index]+word_amount)
                    propbability[index]+=math.log(smooth_prob)
                    index+=1
        max_index=[index for index,item in enumerate(propbability) if item==max(propbability)]
        return possible_label[choice(max_index)]


In [5]:
#Method to train a Standard Naive Bayes Model from a dataset
def train_naive_Bayesian(dataset):
    word_dict=dict()
    total_count=[0,0,0,0]
    label_count=[0,0,0,0]
    for D in dataset:
        analyze_abstract(word_dict,total_count,D[0],D[1])
        label_count[possible_label.index(D[1])]+=1
    return BayseianModel(word_dict,[math.log(x/len(dataset)) if x!=0 else 0 for x in label_count],total_count)

def analyze_abstract(word_dict,total_count,abstract:str, label):
    word_list=abstract.split()
    label_index=possible_label.index(label)
    total_count[label_index]+=len(word_list)
    for word in word_list:
        if word not in word_dict:
            word_dict[word]=[0,0,0,0]
        word_dict[word][label_index]+=1

In [6]:
#Put the Naive Bayes Model to test against a dataset. Output the number of correct guess, the number of incorrect guess and the accuracy
def evaluate(bayesian_model, testset):
    success=0
    fail=0
    for D in testset:
        if(bayesian_model.predict(D[0])==D[1]):
            success+=1
        else:
            fail+=1
    accuracy=success/(success+fail)
    return success,fail,accuracy
#Put the Naive Bayes Model to test against a randomly bootstrap dataset several times. Output the list of accuracy for analyse
def get_test_accuracy_model(model,times):
    accuracy_list=[]
    for i in range(times):
        test_data=[]
        while len(test_data)<800:
            test_data.append(choice(testset))
        accuracy_list.append(evaluate(model,test_data)[2])
    return accuracy_list
#Execute a ten fold cross validation on a Naive Bayes, output the list of validation accuracy for analysis
def ten_fold_cross_validate_NB(data):
    folds=[[],[],[],[],[],[],[],[],[],[]]
    data=list(data)
    while len(data)>0:
        fold_slot=list(range(10))
        while len(fold_slot)>0 and len(data)!=0:
            example=data.pop(randint(0,len(data)-1))
            slot=fold_slot.pop(randint(0,len(fold_slot)-1))
            folds[slot].append(example)
    accuracy_list=[]
    for i in range(10):
        valid_fold=folds[i]
        train_fold=[]
        for f in folds:
            if f!=valid_fold:
                train_fold+=f
        model=train_naive_Bayesian(train_fold)
        evaluation=evaluate(model,valid_fold)
        accuracy_list.append(evaluation[2])
    return accuracy_list

In [7]:
#Bootstrap a training data set
train_data=[]
while len(train_data)<1600:
    train_data.append(choice(trainset))

ten_fold_accuracy_NB=ten_fold_cross_validate_NB(train_data)
print("The result of the ten fold cross validation is:")
print(ten_fold_accuracy_NB)
print(f"The average training error is {sum(ten_fold_accuracy_NB)/10}\n")

#Use the bootstrap training set to train a standard Naive Bayes 
bayesian_model=train_naive_Bayesian(train_data)
#Bootstrap a testing set
test_data=[]
while len(test_data)<800:
    test_data.append(choice(testset))

success,fail,accuracy=evaluate(bayesian_model,test_data)
print(f"There are {success} cases of correct prediction and {fail} cases of incorrect prediction. The accuray of the Naive Bayesian model is {accuracy}\n")

standard_Bayesian=get_test_accuracy_model(bayesian_model,20)
print("The result of 20 test with random dataset of the model is as follows:")
print(standard_Bayesian)
print(f"The accuracy of standard Naive Bayesian modle is {sum(standard_Bayesian)/20}")

The result of the ten fold cross validation is:
[0.9625, 0.95, 0.95625, 0.94375, 0.98125, 0.9625, 0.94375, 0.93125, 0.9875, 0.9375]
The average training error is 0.9556250000000001

There are 736 cases of correct prediction and 64 cases of incorrect prediction. The accuray of the Naive Bayesian model is 0.92

The result of 20 test with random dataset of the model is as follows:
[0.91375, 0.90875, 0.93375, 0.91375, 0.89875, 0.915, 0.93, 0.92375, 0.905, 0.94125, 0.9125, 0.91125, 0.915, 0.925, 0.92, 0.9375, 0.92125, 0.90875, 0.9025, 0.93875]
The accuracy of standard Naive Bayesian modle is 0.9188124999999999


---

### **Part 2.2**: Oversampling Minority class to deal with imbalance data

In [8]:
#Put the Naive Bayes Model to test against a dataset. Output the number of correct guess, the number of incorrect guess and the accuracy for each classes of examples
def detail_evaluation(bayesian_model, testset):
    success=[0,0,0,0]
    fail=[0,0,0,0]
    for D in testset:
        if(bayesian_model.predict(D[0])==D[1]):
            success[possible_label.index(D[1])]+=1
        else:
            fail[possible_label.index(D[1])]+=1
    accuracy=[round(success[i]/(success[i]+fail[i]),2) for i in range(4)]
    total_accuracy=sum(success)/(sum(success)+sum(fail))
    return success,fail,accuracy,total_accuracy

In [9]:
print("The bootstarp training data set has the following classes distribution:")
analyze_label_distribution(train_data)
success,fail,accuracy,total_accuracy=detail_evaluation(bayesian_model,test_data)
print(f"\nThe accuray of the Naive Bayesian model is {total_accuracy}")
print("the number of correct guess, the number of incorrect guess and the accuracy for each classes of examples is in the following:")
print(pd.DataFrame([success,fail,accuracy],index=["success","fail","accuracy"],columns=possible_label))

The bootstarp training data set has the following classes distribution:
                   A      B       E      V
counts         40.00  640.0  872.00  48.00
distributions   0.03    0.4    0.55   0.03

The accuray of the Naive Bayesian model is 0.92
the number of correct guess, the number of incorrect guess and the accuracy for each classes of examples is in the following:
              A       B       E     V
success    8.00  328.00  400.00   0.0
fail      13.00   24.00    6.00  21.0
accuracy   0.38    0.93    0.99   0.0


In [10]:
print("The distribution of classes of examples in the training data set is:")
analyze_label_distribution(train_data)
# Oversample Minority class 
majority=len([d for d in train_data if d[1]=="B" or d[1]=="E"])
oversample_minority=math.floor(majority/2)
ros=RandomOverSampler(random_state=100, sampling_strategy={"A":oversample_minority,"V":oversample_minority})
train_data=ros.fit_resample(train_data,[d[1] for d in train_data])[0]
print("\nThe distribution of classes of examples in the training data set is:")
analyze_label_distribution(train_data)

ten_fold_accuracy_NB=ten_fold_cross_validate_NB(train_data)
print("\nThe result of the ten fold cross validation is:")
print(ten_fold_accuracy_NB)
print(f"The average training error is {sum(ten_fold_accuracy_NB)/10}\n")

# Train the Standard Naive Bayes again with the oversampled training data
bayesian_model_oversample=train_naive_Bayesian(train_data)

success,fail,accuracy,total_accuracy=detail_evaluation(bayesian_model_oversample,test_data)
print(f"The accuray of the Naive Bayesian model is {total_accuracy}")
print("the number of correct guess, the number of incorrect guess and the accuracy for each classes of examples is in the following:")
print(pd.DataFrame([success,fail,accuracy],index=["success","fail","accuracy"],columns=possible_label))

oversample_Bayesian=get_test_accuracy_model(bayesian_model_oversample,20)
print("\nThe result of 20 test with random dataset of the model is as follows:")
print(oversample_Bayesian)
print(f"The accuracy of standard Naive Bayesian modle with oversampled minority class is {sum(oversample_Bayesian)/20}")

The distribution of classes of examples in the training data set is:
                   A      B       E      V
counts         40.00  640.0  872.00  48.00
distributions   0.03    0.4    0.55   0.03

The distribution of classes of examples in the training data set is:
                    A       B       E       V
counts         756.00  640.00  872.00  756.00
distributions    0.25    0.21    0.29    0.25

The result of the ten fold cross validation is:
[0.9768211920529801, 0.9933774834437086, 0.9933774834437086, 0.9933993399339934, 0.9834437086092715, 0.9966887417218543, 0.9966996699669967, 0.9602649006622517, 0.9900990099009901, 0.9966996699669967]
The average training error is 0.9880871199702753

The accuray of the Naive Bayesian model is 0.93875
the number of correct guess, the number of incorrect guess and the accuracy for each classes of examples is in the following:
              A       B       E      V
success   10.00  325.00  400.00  16.00
fail      11.00   27.00    6.00   5.00


In [11]:
stats,pvalue =ttest_ind(standard_Bayesian,oversample_Bayesian)
siglvl=math.ceil(100-abs(stats*2))
print(f"At Least {siglvl}% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm")
print("and standard Naive Bayes algorithem with oversample minority class on the test dataset")

At Least 91% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm
and standard Naive Bayes algorithem with oversample minority class on the test dataset


---

### **Part 2.2**: Switch to Complement Naive Bayes Algorithm

In [12]:
# Complement Naive Bayes algorithem, modified from the standard Naive Bayes algorithm from Part 1
class ComplementBayesainModel():
    def __init__(self,word_weight,label_prob,total_word_count) -> None:
        self.__word_weight=word_weight
        self.__label_prob=label_prob
        self.__total_word_count=total_word_count
    
    @property
    def word_weight(self):
        return self.__word_weight
    
    @property
    def label_prob(self):
        return self.__label_prob
    
    @property
    def total_word_count(self):
        return self.__total_word_count

    def predict(self,abstract:str):
        word_amount=len(self.word_weight)
        word_list=abstract.split()
        propbability=list(self.label_prob)
        for w in word_list:
            if w in self.word_weight:
                index=0
                while index<4:
                    smooth_prob=(sum(self.word_weight[w])-self.word_weight[w][index]+1)/(sum(self.total_word_count)-self.total_word_count[index]+word_amount)
                    propbability[index]-=math.log(smooth_prob)
                    index+=1
        max_index=[index for index,item in enumerate(propbability) if item==max(propbability)]
        return possible_label[choice(max_index)]

In [13]:
#Training and ten fold cross validation method for Complement Naive Bayes algorithm
def train_complement_naive_Bayesian(dataset):
    word_dict=dict()
    total_word_count=[0,0,0,0]
    label_count=[0,0,0,0]
    for D in dataset:
        analyze_abstract(word_dict,total_word_count,D[0],D[1])
        label_count[possible_label.index(D[1])]+=1
    return ComplementBayesainModel(word_dict,[math.log(x/len(dataset)) if x!=0 else 0 for x in label_count],total_word_count)

def ten_fold_cross_validate_CNB(data):
    folds=[[],[],[],[],[],[],[],[],[],[]]
    data=list(data)
    while len(data)>0:
        fold_slot=list(range(10))
        while len(fold_slot)>0 and len(data)>0:
            seed()
            example=data.pop(randint(0,len(data)-1))
            slot=fold_slot.pop(randint(0,len(fold_slot)-1))
            folds[slot].append(example)
    accuracy_list=[]
    for i in range(10):
        valid_fold=folds[i]
        train_fold=[]
        for f in folds:
            if f!=valid_fold:
                train_fold+=f
        model=train_complement_naive_Bayesian(train_fold)
        evaluation=evaluate(model,valid_fold)
        accuracy_list.append(evaluation[2])
    return accuracy_list

In [14]:
# Bootstrap training set
train_data=[]
while len(train_data)<1600:
    train_data.append(choice(trainset))
# Oversample minority class
majority=len([d for d in train_data if d[1]=="B" or d[1]=="E"])
oversample_minority=math.floor(majority/2)
ros=RandomOverSampler(random_state=100, sampling_strategy={"A":oversample_minority,"V":oversample_minority})
train_data=ros.fit_resample(train_data,[d[1] for d in train_data])[0]

ten_fold_accuracy_NB_FS=ten_fold_cross_validate_CNB(train_data)
print("The result of ten fold cross validation is as follows:")
print(ten_fold_accuracy_NB_FS)
print(f"The Complement Naive Bayes algorithm with oversmapling minority class has an average validation accuracy of {sum(ten_fold_accuracy_NB_FS)/10}\n")

CNB_model=train_complement_naive_Bayesian(train_data)

success,fail,accuracy,total_accuracy=detail_evaluation(CNB_model,test_data)
print(f"The accuray of the Complement Naive Bayes algorithm with oversmapling minority class is {total_accuracy}")
print("The accuracy of the Complement Naive Bayes algorithm with oversmapling minority class for each classes are:")
print(pd.DataFrame([success,fail,accuracy],index=["success","fail","accuracy"],columns=possible_label))

oversample_CNB=get_test_accuracy_model(CNB_model,20)
print("\nThe result of 20 test with random dataset of the model is as follows:")
print(oversample_CNB)
print(f"The accuracy of  Complement Naive Bayes algorithm with oversmapling minority class is {sum(oversample_CNB)/20}")

The result of ten fold cross validation is as follows:
[0.9901315789473685, 0.9901315789473685, 0.9901315789473685, 0.9835526315789473, 0.9769736842105263, 0.9802631578947368, 0.9802631578947368, 0.9901315789473685, 0.9703947368421053, 0.993421052631579]
The Complement Naive Bayes algorithm with oversmapling minority class has an average validation accuracy of 0.9845394736842106

The accuray of the Complement Naive Bayes algorithm with oversmapling minority class is 0.9625
The accuracy of the Complement Naive Bayes algorithm with oversmapling minority class for each classes are:
              A       B       E     V
success   16.00  330.00  403.00  21.0
fail       5.00   22.00    3.00   0.0
accuracy   0.76    0.94    0.99   1.0

The result of 20 test with random dataset of the model is as follows:
[0.95875, 0.94625, 0.96125, 0.9475, 0.945, 0.955, 0.95, 0.94875, 0.96625, 0.95125, 0.96375, 0.9425, 0.95625, 0.94875, 0.95375, 0.95875, 0.95375, 0.9425, 0.95625, 0.95625]
The accuracy of  Com

In [15]:
stats,pvalue =ttest_ind(standard_Bayesian,oversample_Bayesian)
siglvl=math.ceil(100-abs(stats*2))
print(f"At Least {siglvl}% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm")
print("and Complementary Naive Bayes algorithem with oversample minority class on the test dataset\n")
stats,pvalue =ttest_ind(oversample_Bayesian,oversample_CNB)
siglvl=math.ceil(100-abs(stats*2))
print(f"At Least {siglvl}% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm")
print("with oversample minority class and Complementary Naive Bayes algorithem with oversample minority class on the test dataset")

At Least 91% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm
and Complementary Naive Bayes algorithem with oversample minority class on the test dataset

At Least 86% confident level, there is a statistical significant difference between the accuracy of standard Naive Bayes algorithm
with oversample minority class and Complementary Naive Bayes algorithem with oversample minority class on the test dataset


---

### **Part 3**: Train and Test to submit Kaggle

In [16]:
# Bootstrap training datas
seed(100)
train_data=[]
while len(train_data)<len(dataset):
    train_data.append(choice(dataset))
# Oversample minority class with Iblearn
majority=len([d for d in train_data if d[1]=="B" or d[1]=="E"])
oversample_minority=math.floor(majority/2)
ros=RandomOverSampler(random_state=100, sampling_strategy={"A":oversample_minority,"V":oversample_minority})
train_data=ros.fit_resample(train_data,[d[1] for d in train_data])[0]
# Train a Complement Naive Bayesian model (see Part 2.2) with training Data 
Submission_model=train_complement_naive_Bayesian(train_data)
# Read tst.csv file and record predicted classes with the model
Predictset=[]
file=open("tst.csv")
test_csv=csv.DictReader(file)
for row in test_csv:
   abstract=row["abstract"]
   Predictset.append([row["id"],Submission_model.predict(abstract)])
file.close()
# Write all predictions model to Predict.csv file
file=open("Predict.csv","w")
file.writelines("id,class\n")
for p in Predictset:
    file.writelines(str.join(",",p)+"\n")
file.close()
